<a href="https://colab.research.google.com/github/aroosaAj/Task2/blob/main/Avengers_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Install libraries
!pip install tensorflow keras numpy matplotlib mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.1 MB/s eta 0:00:00


In [ ]:
!pip install matplotlib-venn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# define data preparation
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)

# prepare iterator
train_it = train_datagen.flow_from_directory('/content/drive/MyDrive/Avengers/train',
                                             class_mode='categorical',
                                             batch_size=64,
                                             target_size=(224, 224),
                                             subset='training')
val_it = train_datagen.flow_from_directory('/content/drive/MyDrive/Avengers/validation',
                                           class_mode='categorical',
                                           batch_size=64,
                                           target_size=(224, 224),
                                           subset='validation')


Found 130 images belonging to 5 classes.
Found 10 images belonging to 5 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 17, 17, 128)     

In [ ]:
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.layers import Dense
from keras.models import Model
from keras.layers import Flatten


# load model without classifier layers
model = VGG16(include_top=False, input_shape=(224, 224, 3))
num_classes = len(train_it.class_indices)

# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(num_classes, activation='softmax')(class1)

# define new model
model = Model(inputs=model.inputs, outputs=output)

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# fit model
history = model.fit(train_it, steps_per_epoch=len(train_it),
                    validation_data=val_it, validation_steps=len(val_it), epochs=50)


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/50
3/3 [==============================] - 190s 52s/step - loss: 2.0757 - accuracy: 0.2385 - val_loss: 2.7364 - val_accuracy: 0.3000
Epoch 2/50
3/3 [==============================] - 171s 44s/step - loss: 2.5002 - accuracy: 0.2538 - val_loss: 1.7302 - val_accuracy: 0.2000
Epoch 3/50
3/3 [==============================] - 170s 44s/step - loss: 1.7711 - accuracy: 0.1769 - val_loss: 1.9265 - val_accuracy: 0.2000
Epoch 4/50
3/3 [==============================] - 171s 45s/step - loss: 1.8838 - accuracy: 0.2231 - val_loss: 1.6809 - val_accuracy: 0.2000
Epoch 5/50
3/3 [==============================] - 171s 44s/step - loss: 1.6451 - accuracy: 0.1846 - val_loss: 1.6822 - val_accuracy: 0.3000
Epoch 6/50
3/3 [==============================] - 170s 84s/step - loss: 1.7366 - accuracy: 0.2000 - val_loss: 1.6637 - val_accuracy: 0.2000
Epoch 7/50
3/3 [==============================] - 171s 44s/step - loss: 1.6188 - accuracy: 0.2385 - val_loss: 1.5939 - val_accuracy: 0.2000
Epoch 8/50
3/3 [====

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# load test images
test_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)
test_it = test_datagen.flow_from_directory('/content/drive/MyDrive/Avengers/test',
                                           class_mode='categorical',
                                           batch_size=64,
                                           target_size=(224, 224))


Found 55 images belonging to 5 classes.


In [ ]:
# save model
model.save('avengers_face_recognition.h5')